In [2]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test['id']

Label_encoder = LabelEncoder()

data['Sex_encoded'] = Label_encoder.fit_transform(data['Sex'])
data.drop('Sex', axis=1, inplace=True)

test['Sex_encoded'] = Label_encoder.fit_transform(test['Sex'])
test.drop('Sex', axis=1, inplace=True)

y = data['Rings']
X = data.drop('Rings', axis = 1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Scale the features for KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Define custom scorer for RMSLE
def rmsle_score(y_true, y_pred):
    rmsle = np.sqrt(mean_squared_log_error(y_true, y_pred))
    return -rmsle  # Negate RMSLE to maximize (since GridSearchCV maximizes by default)

# Make RMSLE scorer
rmsle_scorer = make_scorer(rmsle_score)

# Initialize models
rf_regressor = RandomForestRegressor(random_state=42)
gb_regressor = GradientBoostingRegressor(random_state=42)
knn_regressor = KNeighborsRegressor()


# Define parameter distributions for each model
param_dist_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_dist_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_dist_knn = {
    'n_neighbors': [3, 5, 10, 20],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Perform randomized search for each model
grid_search_rf = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_dist_rf, scoring=rmsle_scorer, cv=5, random_state=42)
grid_search_gb = RandomizedSearchCV(estimator=gb_regressor, param_distributions=param_dist_gb, scoring=rmsle_scorer, cv=5, random_state=42)
grid_search_knn = RandomizedSearchCV(estimator=knn_regressor, param_distributions=param_dist_knn, scoring=rmsle_scorer, cv=5, random_state=42)

# Fit the randomized search objects
grid_search_rf.fit(X_train, y_train)
grid_search_gb.fit(X_train, y_train)
grid_search_knn.fit(X_train_scaled, y_train)

# Get the best estimators
best_rf_regressor = grid_search_rf.best_estimator_
best_gb_regressor = grid_search_gb.best_estimator_
best_knn_regressor = grid_search_knn.best_estimator_

# Train the VotingRegressor with the best estimators
voting_regressor_tuned = VotingRegressor([('rf', best_rf_regressor), ('gb', best_gb_regressor), ('knn', best_knn_regressor)])
voting_regressor_tuned.fit(X_train, y_train)


VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(max_depth=10,
                                                   min_samples_split=5,
                                                   random_state=42)),
                            ('gb',
                             GradientBoostingRegressor(max_depth=5,
                                                       min_samples_split=5,
                                                       n_estimators=200,
                                                       random_state=42)),
                            ('knn',
                             KNeighborsRegressor(n_neighbors=20, p=1,
                                                 weights='distance'))])

In [6]:
predictions = voting_regressor_tuned.predict(X_val)

# Ensure predictions are non-negative
predictions = np.clip(predictions, 0, None)

# Calculate RMSLE
rmsle = np.sqrt(mean_squared_log_error(np.log1p(predictions), np.log1p(y_val)))
print("RMSLE Score:", rmsle)

RMSLE Score: 0.05372231815559575


In [7]:
submission_preds = voting_regressor_tuned.predict(test)

# Ensure submission predictions are non-negative
submission_preds = np.clip(submission_preds, 0, None)

# Round the predictions and convert them to integers
submission_preds_rounded = np.round(submission_preds).astype(int)

In [8]:
df = pd.DataFrame({'id': test_ids.values, 
                   'Rings': submission_preds_rounded
                   })

In [10]:
df.to_csv('submission5.csv', index=False)

In [11]:
# Check if any target value is negative
negative_targets_exist = (df['Rings'] < 0).any()

if negative_targets_exist:
    print("Negative target values exist.")
else:
    print("No negative target values.")


No negative target values.
